In [20]:
import pandas as pd
import csv
import seaborn as sns
import numpy as np
import networkx as nx
import graphviz
import requests
import re
import json

In [21]:
from IPython.display import display
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer, StandardScaler
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.tree import DecisionTreeClassifier, export_graphviz
#from sklearn.ensemble import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from http.server import BaseHTTPRequestHandler, HTTPServer

from bs4 import BeautifulSoup

In [ ]:
#Identify and locate databASE FOR USE
url = "https://people.dbmi.columbia.edu/~friedma/Projects/DiseaseSymptomKB/index.html"
#Scrape data and download table
table = pd.read_html(url, encoding='windows-1252')[0]

In [3]:
#Save CSV file
table.to_csv("disease_symptom_dataset_unprocessed.csv", index=False)

NameError: name 'table' is not defined

In [5]:
#Read CSV File
disease_symptom_dataset_unprocessed = pd.read_csv('disease_symptom_dataset_unprocessed.csv')

#Drop the first row
disease_symptom_dataset_unprocessed = disease_symptom_dataset_unprocessed.drop(0, axis=0)

#Reset Index
disease_symptom_dataset_unprocessed = disease_symptom_dataset_unprocessed.reset_index(drop=True)

#Rename Columns
disease_symptom_dataset_unprocessed.columns = ['disease', 'occurences', 'symptoms']

#Handle Nan values
disease_symptom_dataset_unprocessed = disease_symptom_dataset_unprocessed.fillna(method='ffill')

C:\Users\user\AppData\Local\Temp\ipykernel_6088\1075060269.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  disease_symptom_dataset_unprocessed = disease_symptom_dataset_unprocessed.fillna(method='ffill')


In [6]:
#Function to convert
def listToString(s):
    #Initialize empty String
    str1 = " "
    #Return string
    return (str1.join(s))
    
#Process disease and symptom columns
def remove_umls_code(data):
    #Regular expression pattern to match UMLS code
#    pattern = r'UMLS:C[0-9]+_(.*)'
    #Replace UMLS codes with the captured group (text after _)
#    data = re.sub(pattern,r'\1', data)
#    return data
    data_ = []
    items = data.replace('^','_').split('_')
    i = 1
    for item in items:
        if (i % 2 == 0):
            data_.append(item)
        i += 1
    return data_

In [7]:
diseases = []
symptoms = []
occurences = 0
disease_symptoms = defaultdict(list) #or d.setdefault(key, []).append(disease_symptoms)
disease_symptoms_occurences = {}
disease_symptom_df = pd.DataFrame([], columns=["diseases", "symptoms"])
for index, row in disease_symptom_dataset_unprocessed.iterrows():
    
    if (row['disease']!="\xc2\xa0") and (row['disease']!=""):
        diseases = remove_umls_code(row['disease'])    
    occurences = (row['occurences'] if (row['occurences'] !="\xc2\xa0") and (row['occurences']!="") else[])
    #if (row['symptoms']!="\xc2\xa0) and (row['symptoms']!=""):
     #   symptoms = remove_umls_code(row['symptoms'])
    symptoms = [] if (row['symptoms']!="\xc2\xa0") and (row['symptoms']!="") else None
                      
    symptoms = remove_umls_code(row['symptoms']) if symptoms == [] else symptoms
    disease_symptom_df = disease_symptom_df._append(
        {
            "diseases": listToString(remove_umls_code(row['disease']) if (row['disease']!="\xc2\xa0") and (row['disease']!="") 
                                     else []), 
            "symptoms": listToString(remove_umls_code(row['symptoms']) if (row['symptoms']!="\xc2\xa0") and (row['disease']!="") 
                                     else[])
        }, ignore_index=True)
    for d in diseases:
            for s in symptoms:
                disease_symptoms.setdefault(d, []).append(s)
            disease_symptoms_occurences[d] = occurences

In [8]:
#Save the cleaned dataset to a CSV file
disease_symptom_dataset_unprocessed.to_csv("disease_symptom_dataset_processed.csv")
disease_symptom_dataset_processed = pd.read_csv('disease_symptom_dataset_processed.csv')

#Save disease symptoms list to a csv file
disease_symptoms_data = pd.DataFrame.from_dict(disease_symptoms.items())
disease_symptoms_data.to_csv('disease_symptoms.csv')

#Save disease symtom occurences list to a CSV file
disease_symptoms_occurences_data = pd.DataFrame.from_dict(disease_symptoms_occurences.items())
disease_symptoms_occurences_data.to_csv('disease_occurences.csv')

In [9]:
disease_symptom_df.head()
disease_symptom_df.to_csv('disease-symptom_df.csv')

In [ ]:
disease_symptom_df.to_csv('disease_symptoms_network.csv')
#0. Figure size
plt.figure(figsize=(160, 160)).set_facecolor("w")
# 1. Create a graph
disease_symptom_network = nx.from_pandas_edgelist(disease_symptom_df, source='diseases', target='symptoms')
# 2. Create a layout ofr our nodes
layout = nx.spring_layout(disease_symptom_network, iterations=200)
# 3. Draw the parts we want
nx.draw_networkx_edges(disease_symptom_network, layout, edge_color='#AAAAAA')

#Symptoms
symptoms_n = [node for node in disease_symptom_network.nodes() if node in disease_symptom_df.symptoms.unique()]
symptoms_size = [disease_symptom_network.degree(node)*200 for node in disease_symptom_network.nodes() if node in disease_symptom_df.symptoms.unique()]
nx.draw_networkx_nodes(disease_symptom_network, layout, nodelist=symptoms_n, node_size=symptoms_size, node_color='#27ae60')

#Diseases
disease_n = [node for node in disease_symptom_network.nodes() if node in disease_symptom_df.diseases.unique()]
disease_size = [disease_symptom_network.degree(node)*200 for node in disease_symptom_network.nodes() if node in disease_symptom_df.diseases.unique()]
nx.draw_networkx_nodes(disease_symptom_network, layout, nodelist=disease_n, node_size=disease_size, node_color='#e67e22')

#Label Network
symptoms_dict = dict(zip(symptoms_n, symptoms_n))
diseases_dict = dict(zip(disease_n, disease_n))
nx.draw_networkx_labels(disease_symptom_network, layout, labels=symptoms_dict)
nx.draw_networkx_labels(disease_symptom_network, layout, labels=diseases_dict)

#Turn off axis
plt.axis('off')
#Titel
plt.title("DISEASE-SYMPTOMS-NETWORK")
# 5. Save the plot
plt.savefig('disease_symptom_network.png')
# 6. Show
#plt.show()

In [11]:
#disease_symptom_network.nodes.data()
#@app.route('/get_common_target_connected_nodes', methods=['POST']
def get_common_target_connected_nodes(graph, nodes):
#    nodes = request.get_json()['nodes']
    if len(nodes) == 1:
        return list(graph.neighbors(nodes[0]))
    common_target_nodes = set()
    common_target_nodes = set(graph.neighbors(nodes[0]))
    for node in nodes[1:]:
        common_target_nodes &= set(graph.neighbors(node))
    return list(common_target_nodes)
    #common_neighbors = set(graph.neighbors(nodes[0]))
    #for node in nodes[1:]:
      #  common_neighbors &= set(graph.neighbors(node))
     #   common_nodes_in_source = [node for node in common_neighbors if node in nodes]
    #return common_nodes_in_source
    
    #for neighbor in G.neighbors(node):
     #   print(neighbor)
      #  if neighbor not in seen:
       #     seen.add(neighbor)
        #    fetch_connected_nodes(G, neighbor, seen)
    #return seen

def preprocess_input(input_str):
    #Remove any leading or trailing white spaces
    input_str = input_str.strip()
    #Split theinput into a list of nodes
    nodes = input_str.split(',')
    return nodes

def validate_input(graph, nodes):
    valid_nodes = []
    for node in nodes:
        if node in graph:
            valid_nodes.append(node)
        else:
            # If the node is not in the graph, try splitting it by comma
            split_nodes = node.split(' ')
            for split_node in split_nodes:
                split_node = split_node.strip()
                if split_node in graph:
                    valid_nodes.append(split_node)
                else:
                    #If the node is still not valid, print error message
                    print(f"Error: The symptom {node} is not registered on our database, perhaps it could be a typo.")
    return valid_nodes

In [12]:
from flask import Flask, request, jsonify, render_template, url_for
import json

In [29]:
def process_data(nodes):
    #Preprocess the input
    preprocessed_nodes = preprocess_input(nodes)
    #Validate the input
    valid_nodes = validate_input(disease_symptom_network, preprocessed_nodes)
    #Get the common taret nodes
    common_target_nodes = get_common_target_connected_nodes(disease_symptom_network, valid_nodes)
    return common_target_nodes

NameError: name 'self' is not defined

In [ ]:
def main():
    #Get user input
    #input_str = input("Please list all symptoms(Seperated by commas):")
    #Preprocess the input
    nodes = preprocess_input(input_str)
    #Validate the input
    valid_nodes = validate_input(disease_symptom_network, nodes)
    if valid_nodes:
        #Get the common taret nodes
        common_target_nodes = get_common_target_connected_nodes(disease_symptom_network, valid_nodes)
    
    #Print the result
    if common_target_nodes:
        if len(common_target_nodes) == 1:
            print("The most ailment based off of your symptoms is: ", common_target_nodes, "Please visit a certified health professional for medical advice")
        else:
            print("Possible ailments based off of your symptoms are: ", common_target_nodes)
    else:
        break
    break

if __name__=="__main__":
    main()